In [1]:
from class_definitions import *

Wines[:5]

[Wine: Zweigelt 2016 Bauer, Source: name-search,
 Wine: Truffle Hound 2017 Unico Zelo, Source: name-search,
 Wine: Sforzato di Valtellina 2013 Alfio Mozzi, Source: name-search,
 Wine: Cabernet Sauvignon 2017 Oracle Cellars, Source: name-search,
 Wine: Spatburgunder Kabinett 2017 Koehler-Ruprecht, Source: name-search]

In [2]:
from vivino_db import database

Engine(sqlite://///Users/schlinkertc/code/wine_project/wine/vivino_db/vivino_wines.db)


In [3]:
db = database.db

In [4]:
vintages_df = pd.read_sql_table('vintages',db.db_engine)
wines_df = pd.read_sql('select * from wines',db.db_engine)
styles_df = pd.read_sql_table("styles",db.db_engine)
regions_df = pd.read_sql_table("regions",db.db_engine)
wineries_df = pd.read_sql_table("wineries",db.db_engine)

In [5]:
db.db_engine.table_names()

['regions', 'styles', 'vintages', 'wineries', 'wines']

In [100]:
wines_df.columns

Index(['id', 'name', 'seo_name', 'type_id', 'is_natural', 'region_id',
       'review_status', 'winery_id', 'statistics_status',
       'statistics_ratings_count', 'statistics_ratings_average',
       'statistics_labels_count', 'statistics_vintages_count', 'hidden',
       'description', 'non_vintage', 'alcohol', 'created_at', 'is_first_wine',
       'style_id', 'sweetness_id'],
      dtype='object')

In [86]:
styles_df.columns

Index(['id', 'seo_name', 'regional_name', 'varietal_name', 'name', 'body',
       'body_description', 'acidity', 'acidity_description', 'country_code',
       'country_name', 'country_native_name', 'country_seo_name',
       'country_regions_count', 'country_users_count', 'country_wines_count',
       'country_wineries_count', 'wine_type_id', 'region_id',
       'background_image_location', 'description', 'blurb'],
      dtype='object')

### Fuzzy matching from KCW to vivino

In [6]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [7]:
vivino_vintages = vintages_df['name'].to_list()

In [87]:
stmt = """
SELECT 
    v.name as vivino_wine,
    ws.name as winery,
    r.name as region,
    s.name as style,
    s.varietal_name,
    s.regional_name,
    s.body,
    s.acidity,
    s.description as style_description,
    v.statistics_ratings_count as ratings_count,
    v.statistics_ratings_average as ratings_average,
    v.description as vintage_description,
    v.ranking_region_rank / v.ranking_region_total as ranking_region,
    v.ranking_global_rank / v.ranking_global_total as ranking_global,
    v.wine_facts_contains_added_sulfites as added_sulfites,
    v.wine_facts_certified_biodynamic as biodynamic,
    w.description as wine_description
FROM 
    vintages v
INNER JOIN 
    wines w on v.wine_id==w.id
INNER JOIN 
    wineries ws on w.winery_id=ws.id
INNER JOIN 
    styles s on s.id = w.style_id
INNER JOIN 
    regions r on r.id = w.region_id
"""

In [88]:
pd.read_sql(stmt,db.db_engine)

,vivino_wine,winery,region,style,varietal_name,regional_name,body,acidity,style_description,ratings_count,ratings_average,vintage_description,ranking_region,ranking_global,added_sulfites,biodynamic
0,West+Wilder White,West+Wilder,California,Californian White Blend,White Blend,Californian,3,3,None,12,3.2,,0.381230,0.365556,NaN,NaN
1,Ostatu Rioja Tinto,Ostatu,Rioja Alavesa,Spanish Rioja Red,Rioja Red,Spanish,4,3,Rioja is arguably Spain's top wine region and ...,944,3.4,,0.317237,0.182369,NaN,NaN
2,Le Pianate Un Paio Rosso,Le Pianate,Umbria,Central Italy Red,Red,Central Italy,3,3,Perhaps the greatest wines in the Chianti area...,34,4.3,,0.038699,0.037175,NaN,NaN
3,Dirupi Vino Sbagliato Sforzato di Valtellina,Dirupi,Sforzato della Valtellina,Italian Nebbiolo,Nebbiolo,Italian,5,3,None,296,4.4,,0.001062,0.002333,NaN,NaN
4,Produttori di Portacomaro Barolo,Produttori di Portacomaro,Barolo,Italian Barolo,Barolo,Italian,5,3,Barolo is a delicious red wine from the Piedmo...,167,3.8,,0.091022,0.067287,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,Vall de Baldomar Petit Baldomà Tinto,Vall de Baldomar,Costers del Segre,Spanish Red,Red,Spanish,4,3,The savory red wines of Spain are brooding and...,463,3.3,,0.288680,0.215776,NaN,NaN
970,Schola Sarmenti Fiano Bianco,Schola Sarmenti,Salento,Southern Italy White,White,Southern Italy,3,3,Italy is responsible for some of the finest wi...,429,3.6,,0.319521,0.262504,NaN,NaN
971,C&H Fournaise Blanc de Blancs Champagne,C&H Fournaise,Champagne,French Champagne,Champagne,French,3,3,While there are many sparkling wine regions ar...,0,0.0,,NaN,NaN,NaN,NaN
972,Casillero del Diablo Sauvignon Blanc (Reserva),Casillero del Diablo,Casablanca Valley,Chilean Sauvignon Blanc,Sauvignon Blanc,Chilean,3,3,Chile is known for making some of the most pop...,18554,3.4,,0.199740,0.182944,NaN,NaN


In [20]:
wine_matches = []
problems = []
for wine in Wines:
    search_string = "".join([x+' ' for x in wine.search_terms]).strip(' ')
    matches = [(x,fuzz.token_sort_ratio(search_string,x),wine) for x in vivino_vintages]
    best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
    if best_match[1] > 60:
        wine_matches.append(best_match)
    else:
        matches = [(x,fuzz.token_sort_ratio(wine.name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        if best_match[1] > 60:
            wine_matches.append(best_match)
        else:
            problems.append(best_match)

In [26]:
wine_matches = []
for wine in Wines:
    if len(wine.wineSearcher.keys())>2:
        wine_name = wine.wineSearcher['name']
        matches = [(x,fuzz.token_sort_ratio(wine_name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        wine_matches.append(best_match)
    else:
        matches = [(x,fuzz.token_sort_ratio(wine.name,x),wine) for x in vivino_vintages]
        best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
        if best_match[1] > 60:
            wine_matches.append(best_match)
        else:
            search_string = "".join([x+' ' for x in wine.search_terms]).strip(' ')
            matches = [(x,fuzz.token_sort_ratio(search_string,x),wine) for x in vivino_vintages]
            best_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
            wine_matches.append(best_match)


In [30]:
problems = list(filter(lambda x: x[1]<70, wine_matches))

In [31]:
len(problems)

66

In [11]:
problem_ids = []
for problem in problems:
    problem_ids.extend([vw['wine_link'].split('/')[-1] for vw in problem[-1].vivino])

In [12]:
len(problem_ids)

228

In [52]:
from vivino_scraper import *

In [55]:
import time
still_not_added = []
for wine_id in ["26696058"]:
    try:
        result = scrape_vivino(wine_id)
        joblib.dump(result,storage_directory+str(wine_id))
        print('*')
        time.sleep(5)
    except:
        still_not_added.append(wine_id)
        continue

*


In [33]:
wineSearcher_errors = []
for wine in Wines:
    if len(wine.wineSearcher.keys())==2:
        wineSearcher_errors.append(wine)

In [40]:
def match_toVivino(wine):
    vintages_df = pd.read_sql_table('vintages',db.db_engine)
    wine_name = wine.wineSearcher['name']
    search_string = "".join([x+' ' for x in wine.search_terms]).strip(' ')
    
    matches = [(x,fuzz.token_sort_ratio(wine_name,x),wine) for x in vivino_vintages]
    name_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
    
    matches = [(x,fuzz.token_sort_ratio(search_string,x),wine) for x in vivino_vintages]
    searchString_match = sorted(matches,key=lambda x: x[1],reverse=True)[0]
    
    if name_match[1]>searchString_match[1]:
        return name_match
    else:
        return searchString_match
    

In [73]:
s = match_toVivino(wineSearcher_errors[0])

In [74]:
s

('Stefan Bauer Zweigelt', 80, Wine: Zweigelt 2016 Bauer, Source: name-search)

In [44]:
s.replace(' ','+')

'Stefan+Bauer+Zweigelt'

In [63]:
def update_wineSearcher_fromVivino(wine):
    vivino_match = match_toVivino(wine)
    search_terms=vivino_match[0].replace(' ',"+")

    url = f'https://www.wine-searcher.com/find/{search_terms}'

    wine_dict = {'url':url}

    page = requests.get(url,headers=Wine.headers)
    if page.status_code != 200:
        return page.status_code
    bs = BeautifulSoup(page.content,'lxml')

    wine_dict['name'] = bs.find(
        'h1',attrs={'class':'wine'}).\
            getText(strip=True)

    wine_info_panel = bs.find('div',attrs={"class":'wine-info-panel'})
    if wine_info_panel==None:
        print(f"failure{wine}")
        return wine_dict
    wine_info = []

    for div in wine_info_panel.find_all('div',attrs={'class':'dtc'}):
        wine_info.append(
            [x.getText() for x in div.find_all(
                'span',attrs={'class':"dtlbl sidepanel-text"}
            ) 
             if type(x)==bs4.element.Tag])

    wine_info = [
        item.split('\n') for sublist in wine_info for item in sublist
    ]
    wine_info = [
        item.strip() for sublist in wine_info 
        for item in sublist if item not in  ['',' ']
    ]

    wine_dict_keys = [
        'Producer','Region/Appellation',
        'Grape/Blend','Food Suggestion',
        'Wine Style','Alcohol Content','Notes'
    ]

    for key in wine_dict_keys:
        try:
            key_index = wine_info.index(key)
            wine_dict[key]=wine_info[key_index+1]
        except:
            wine_dict[key]='null'
    
    wine.wineSearcher = wine_dict
    
    joblib.dump(wine,f"Wines/name-search/{wine.name}.pkl")
    print('success')

In [64]:
import time

In [65]:
for wine in wineSearcher_errors[3:]:
    update_wineSearcher_fromVivino(wine)
    time.sleep(15)

failureWine: Aloxe Corton "Les Citernes" 2015 Bidault, Source: name-search
failureWine: Jacquère Cuvée Baujues 2016 St. Germain, Source: name-search
failureWine: Bugey Cerdon Rosé Demi-Sec NV Dentelle, Source: name-search
failureWine: Refošk Baracija 2017 Clai Wines, Source: name-search


KeyboardInterrupt: 

### Bringing it all together

In [18]:
vintages_df.iloc[vintages_df[vintages_df['name']==least_confident].index[0]]

index                                                              520
id                                                           159955768
seo_name                              gassier-elevae-rose-d-elevage-uv
name                                     Gassier Elevae Rosé d'Elevage
statistics_status                                               Normal
statistics_ratings_count                                            10
statistics_ratings_average                                         4.4
statistics_labels_count                                              4
statistics_reviews_count                                             0
wine_id                                                        7134802
description                                                           
ratings_distribution_1                                               0
ratings_distribution_2                                               0
ratings_distribution_3                                               1
rating

In [41]:
wines_df.iloc[wines_df[wines_df['id']==2237206].index[0]]

index                                           29
id                                         2237206
name                                      Zweigelt
seo_name                                  zweigelt
type_id                                          1
is_natural                                       0
region_id                                      869
review_status                                    2
winery_id                                    42667
statistics_status                           Normal
statistics_ratings_count                        26
statistics_ratings_average                     3.5
statistics_labels_count                        112
statistics_vintages_count                       12
hidden                                           0
description                                       
non_vintage                                      0
alcohol                                         13
created_at                    2014-03-09T08:55:09Z
is_first_wine                  

In [15]:
styles_df.iloc[styles_df[styles_df['id']==255].index[0]]

index                                                                        8
id                                                                         255
seo_name                                                     austrian-zweigelt
regional_name                                                         Austrian
varietal_name                                                         Zweigelt
name                                                         Austrian Zweigelt
body                                                                         4
body_description                                                   Full-bodied
acidity                                                                      3
acidity_description                                                       High
country_code                                                                at
country_name                                                           Austria
country_native_name                                 

In [16]:
regions_df.iloc[regions_df[regions_df['id']==869].index[0]]

index                                28
id                                  869
name                             Wagram
name_en                                
seo_name                      donauland
country_code                         at
country_name                    Austria
country_native_name          Österreich
country_seo_name                austria
country_regions_count                24
country_users_count              147508
country_wines_count               43242
country_wineries_count             4240
background_image_location          None
Name: 28, dtype: object

In [17]:
wineries_df[wineries_df['id']==42667]

,index,id,name,seo_name,status,review_status,statistics_ratings_count,statistics_ratings_average,statistics_labels_count,statistics_wines_count
28,28,42667,Stefan Bauer,stefan-bauer,0,Completed,355,3.9,1264,17


In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [19]:
import time
def selenium_scrape(wine_id):
    url = f"https://vivino.com/wines/{str(wine_id)}"
    
    chrome_driver_path = "/Users/schlinkertc/code/chromedriver"
    driver = webdriver.Chrome(executable_path=chrome_driver_path)
    
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, 1200)")
    driver.execute_script("window.scrollTo(0, 1800)")
    time.sleep(2)
    master_wine_page = driver.find_element_by_id('master-wine-page-app')
    
    driver.set_window_size(width=742,height=983)
    a_tags = driver.find_elements_by_tag_name('a')
    
    for a in a_tags:
        a_class = a.get_attribute('class')
        if 'toggleShowAll' in a_class:
            show_all = a
            break

    a.click()
#     text = master_wine_page.text
#     notes = [x for x in text.split('\n') if 'mentions of' in x]
    
    svgs = master_wine_page.find_elements_by_tag_name('svg')

    tasteNotes = []
    for s in svgs:
        try:
            svg_class = s.get_attribute('class')
            if 'tasteNote' in svg_class:
                tasteNotes.append(s)
        except:
            continue 

    notes = []
    note_tags = []
    for i in range(len(tasteNotes)):
        tasteNotes[i].click()
        time.sleep(5)
        spans = driver.find_elements_by_tag_name('span')
        for span in spans:
            try:
                span_class = span.get_attribute('class')
                if 'tasteReviews__capitalizedTastes' in span_class:
                    note_texts.append(span.text)
            except:
                continue
        divs = driver.find_elements_by_tag_name('div')
        
        
        for div in divs:
            try:
                div_class = div.get_attribute('class')
                if 'noteTag__name' in div_class:
                    note_tags.append({'index':i,'tag':div.text})
            except:
                continue
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        
      ## This is to get the indicator bars. not ready yet  
#     spans = [x for x in master_wine_page.find_elements_by_tag_name('span')]
#     styles = []
#     for span in spans:
#         try:
#             style = span.get_attribute('style')
#             if style.startswith('width'):
#                 styles.append(style)
#         except:
#             continue
    driver.quit()
    indexed_notes = [{'index':n,'note':note} for n,note in enumerate(notes)]
    
    for tag in note_tags:
        for note in indexed_notes:
            if note['index']==tag['index']:
                tag['note']=note['note']
                
    return {wine_id:note_tags}

In [20]:
selenium_scrape(4470482)

{4470482: [{'index': 0, 'tag': 'plum'},
  {'index': 0, 'tag': 'blackcurrant'},
  {'index': 1, 'tag': 'prune'},
  {'index': 2, 'tag': 'violet'},
  {'index': 3, 'tag': 'chard'},
  {'index': 3, 'tag': 'red beet'},
  {'index': 4, 'tag': 'chocolate'}]}

In [165]:
def scrape_wineSearcher_fromVivino(seo_name):
    seo_name = seo_name.strip("-uv").replace('-','+')
    url = f'https://www.wine-searcher.com/find/{seo_name}'

    wine_dict = {'url':url}

    page = requests.get(url,headers=Wine.headers)
    if page.status_code != 200:
        return page.status_code
    bs = BeautifulSoup(page.content,'lxml')

    wine_dict['name'] = bs.find(
        'h1',attrs={'class':'wine'}).\
            getText(strip=True)

    wine_info_panel = bs.find('div',attrs={"class":'wine-info-panel'})
    if wine_info_panel==None:
        return wine_dict
    wine_info = []

    for div in wine_info_panel.find_all('div',attrs={'class':'dtc'}):
        wine_info.append(
            [x.getText() for x in div.find_all(
                'span',attrs={'class':"dtlbl sidepanel-text"}
            ) 
             if type(x)==bs4.element.Tag])

    wine_info = [
        item.split('\n') for sublist in wine_info for item in sublist
    ]
    wine_info = [
        item.strip() for sublist in wine_info 
        for item in sublist if item not in  ['',' ']
    ]

    wine_dict_keys = [
        'Producer','Region/Appellation',
        'Grape/Blend','Food Suggestion',
        'Wine Style','Alcohol Content','Notes'
    ]

    for key in wine_dict_keys:
        try:
            key_index = wine_info.index(key)
            wine_dict[key]=wine_info[key_index+1]
        except:
            wine_dict[key]='null'

    return wine_dict

In [166]:
scrape_wineSearcher_fromVivino("stefan-bauer-zweigelt-uv")

{'url': 'https://www.wine-searcher.com/find/stefan+bauer+zweigelt',
 'name': 'Stefan Bauer Zweigelt, Wagram, Austria',
 'Producer': 'Stefan Bauer',
 'Region/Appellation': 'Wagram',
 'Grape/Blend': 'Zweigelt',
 'Food Suggestion': 'Pork',
 'Wine Style': 'Red - Savory and Classic',
 'Alcohol Content': 'null',
 'Notes': 'null'}

In [144]:
def compile_data(wines):
    #wineSearcher 
    records = []
    for wine in wines:
        record = wine.wineSearcher
        record['instance']=wine
        records.append(record)
    df = pd.DataFrame.from_records(records)
    df.drop(columns=['url','Notes'],inplace=True)
    
    #vivino
    vivino_records = []
    for wine in wines:
        vivino_match = match_toVivino(wine)
        stmt = f"""
        SELECT 
            v.name as vivino_wine,
            ws.name as winery,
            r.name as region,
            s.name as style,
            s.varietal_name,
            s.regional_name,
            s.body,
            s.acidity,
            s.description as style_description,
            v.statistics_ratings_count as ratings_count,
            v.statistics_ratings_average as ratings_average,
            v.description as vintage_description,
            v.ranking_region_rank / v.ranking_region_total as ranking_region,
            v.ranking_global_rank / v.ranking_global_total as ranking_global,
            v.wine_facts_contains_added_sulfites as added_sulfites,
            v.wine_facts_certified_biodynamic as biodynamic,
            w.description as wine_description
        FROM 
            vintages v
        INNER JOIN 
            wines w on v.wine_id==w.id
        INNER JOIN 
            wineries ws on w.winery_id=ws.id
        INNER JOIN 
            styles s on s.id = w.style_id
        INNER JOIN 
            regions r on r.id = w.region_id
        WHERE
            vivino_wine = "{vivino_match[0]}"
        """
        resultproxy = db.db_engine.execute(stmt)

        d, a = {}, []
        for rowproxy in resultproxy:
            # rowproxy.items() returns an array like [(key0, value0), (key1, value1)]
            for column, value in rowproxy.items():
                # build up the dictionary
                d = {**d, **{column: value}}
            a.append(d)
        try:
            vivino_record = a[0]
            vivino_record['instance']=vivino_match[-1]
            vivino_records.append(vivino_record)
        except:
            continue 
    return df,vivino_records

In [145]:
df,v_df = compile_data(Wines)

In [146]:
v_df =  pd.DataFrame.from_records(v_df)

In [147]:
df.shape

(237, 8)

In [148]:
v_df.shape

(161, 18)

In [150]:
data = pd.merge(left=df,right=v_df,on='instance')


In [154]:
data.columns

Index(['name', 'Producer', 'Region/Appellation', 'Grape/Blend',
       'Food Suggestion', 'Wine Style', 'Alcohol Content', 'instance',
       'vivino_wine', 'winery', 'region', 'style', 'varietal_name',
       'regional_name', 'body', 'acidity', 'style_description',
       'ratings_count', 'ratings_average', 'vintage_description',
       'ranking_region', 'ranking_global', 'added_sulfites', 'biodynamic',
       'wine_description'],
      dtype='object')

In [155]:
data[['name','vivino_wine']]

,name,vivino_wine
0,"Stefan Bauer Zweigelt, Wagram, Austria",Stefan Bauer Zweigelt
1,Sforzato di Valtellina 2013 Alfio Mozzi,Plozza Sforzato di Valtellina
2,Cabernet Sauvignon 2017 Oracle Cellars,Oracle Cellars Cabernet Sauvignon
3,Koehler-Ruprecht Spatburgunder Kabinett Trocke...,Koehler-Ruprecht Spätburgunder Kabinett Trocken
4,Sauvignon Blanc 2019 Tablelands,Tablelands Sauvignon Blanc
...,...,...
156,L'Effet Papillon Blanc 2019 Roc des Anges,Le Roc des Anges L'Effet Papillon Blanc
157,"Soave Classico ""Otto"" 2018 Prá",Prà Otto Soave Classico
158,Tai Rosso 2018 Rezzadore,Rezzadore Tai Rosso
159,2019 rose cotes de provence gassier,Gassier La Santonnière Côtes de Provence Rosé


In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161 entries, 0 to 160
Data columns (total 25 columns):
name                   161 non-null object
Producer               122 non-null object
Region/Appellation     137 non-null object
Grape/Blend            137 non-null object
Food Suggestion        131 non-null object
Wine Style             131 non-null object
Alcohol Content        89 non-null object
instance               161 non-null object
vivino_wine            161 non-null object
winery                 161 non-null object
region                 161 non-null object
style                  161 non-null object
varietal_name          161 non-null object
regional_name          161 non-null object
body                   161 non-null int64
acidity                161 non-null int64
style_description      122 non-null object
ratings_count          161 non-null int64
ratings_average        161 non-null float64
vintage_description    161 non-null object
ranking_region         160 non-null fl